In [1]:
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import xgboost as xgb
import cupy as cp
import numpy as np
import json

In [2]:
def train_test_split_by_date(x, y, test_size=0.2):
    """
    Split the input data into training and test sets by date.
    The test set should contain the newest test_size proportion of the data.
    """
    df = pd.concat([x, y], axis=1)
    df = df.sort_values(by='author_date_unix_timestamp')
    split_index = int((1 - test_size) * len(df))
    x_train = df.iloc[:split_index, :-1]
    y_train = df.iloc[:split_index, -1]
    x_test = df.iloc[split_index:, :-1]
    y_test = df.iloc[split_index:, -1]
    return x_train, x_test, y_train, y_test


In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, roc_auc_score

def hyperparameter_optimization(x_train, y_train, n_splits=5, n_trials=50):
    """
    Optimize the hyperparameters of the XGBoost model using cross-validation.

    Parameters:
      - x_train: Training features (pandas DataFrame)
      - y_train: Training labels (pandas Series)
      - n_splits: Number of cross-validation splits (default: 5)
      - n_trials: Number of hyperparameter search trials (default: 50)

    Returns:
      - The best hyperparameters found by Optuna.
    """
    # Convert the entire training set to NumPy arrays once.
    x_train_np = x_train.to_numpy()
    y_train_np = y_train.to_numpy().ravel()

    # Convert the NumPy arrays to GPU arrays (cupy.ndarray).
    x_train_gpu = cp.asarray(x_train_np)
    y_train_gpu = cp.asarray(y_train_np)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    def objective(trial):
        # https://xgboosting.com/most-important-xgboost-hyperparameters-to-tune/
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300), # Number of boosting rounds, 1000 is too much with little no improvement
            'min_child_weight': trial.suggest_float('min_child_weight', 1.0, 7.0, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 20),
            'random_state': 42,
            'n_jobs': -1
        }

        early_stopping_rounds = 50
        
        cv_scores = []
        # Perform stratified K-fold cross-validation using pre-converted arrays.
        for train_idx, valid_idx in skf.split(x_train_np, y_train_np):
            # Slice the pre-converted GPU arrays.
            X_tr_gpu = x_train_gpu[train_idx]
            y_tr_gpu = y_train_gpu[train_idx]
            X_val_gpu = x_train_gpu[valid_idx]
            # Use the original NumPy array for validation labels.
            y_val = y_train_np[valid_idx]
            
            # Configure and train the model.
            model = XGBClassifier(
                **params,
                early_stopping_rounds=early_stopping_rounds,
                tree_method="hist",  # GPU-optimized training
                device="cuda"        # use GPU for training and prediction
            )
            model.fit(X_tr_gpu,
                        y_tr_gpu,
                        eval_set=[(X_val_gpu, y_val)],
                        verbose=False
                      )
            
            # Predict on the validation fold.
            y_pred_prob = model.predict_proba(X_val_gpu)[:, 1]
            y_pred_cpu = cp.asnumpy(y_pred_prob)
            
            # Compute ROC AUC score for the current fold
            fold_score = roc_auc_score(y_val, y_pred_cpu)
            cv_scores.append(fold_score)
        
        # Return the mean ROC AUC across folds.
        return np.mean(cv_scores)
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Free GPU memory to clear up space for subsequent computations.
    cp._default_memory_pool.free_all_blocks()

    return study.best_params


In [4]:
# all columns in the dataset
INFO_COLUMNS = [
                "type", #"linked", "fileschanged",
                "ndev", "age", "exp", "rexp", "sexp",
                "glm_probability", "classification",
                "time_of_day", "day_of_week", "is_weekend",
                "author_experience", "author_ownership",
                "hash","file", "author_date_unix_timestamp"]

Y_COLUMN = ['contains_bug']

X_COLUMNS = ["d_cbo","d_cboModified","d_fanin",
             "d_fanout","d_wmc","d_dit","d_noc",
             "d_rfc","d_lcom","d_lcom*","d_tcc",
             "d_lcc","d_totalMethodsQty","d_staticMethodsQty",
             "d_publicMethodsQty","d_privateMethodsQty","d_protectedMethodsQty",
             "d_defaultMethodsQty","d_visibleMethodsQty","d_abstractMethodsQty",
             "d_finalMethodsQty","d_synchronizedMethodsQty","d_totalFieldsQty",
             "d_staticFieldsQty","d_publicFieldsQty","d_privateFieldsQty",
             "d_protectedFieldsQty","d_defaultFieldsQty","d_finalFieldsQty",
             "d_synchronizedFieldsQty","d_nosi","d_loc","d_returnQty","d_loopQty",
             "d_comparisonsQty","d_tryCatchQty","d_parenthesizedExpsQty","d_stringLiteralsQty",
             "d_numbersQty","d_assignmentsQty","d_mathOperationsQty","d_variablesQty",
             "d_maxNestedBlocksQty","d_anonymousClassesQty","d_innerClassesQty",
             "d_lambdasQty",
             #"d_uniqueWordsQty",
             "d_modifiers",
             #"d_logStatementsQty",
             "cbo","cboModified","fanin","fanout","wmc","dit","noc","rfc","lcom","lcom*",
             "tcc","lcc","totalMethodsQty","staticMethodsQty","publicMethodsQty",
             "privateMethodsQty","protectedMethodsQty","defaultMethodsQty",
             "visibleMethodsQty","abstractMethodsQty","finalMethodsQty",
             "synchronizedMethodsQty","totalFieldsQty","staticFieldsQty",
             "publicFieldsQty","privateFieldsQty","protectedFieldsQty",
             "defaultFieldsQty","finalFieldsQty","synchronizedFieldsQty",
             "nosi","loc","returnQty","loopQty","comparisonsQty",
             "tryCatchQty","parenthesizedExpsQty","stringLiteralsQty",
             "numbersQty","assignmentsQty","mathOperationsQty",
             "variablesQty","maxNestedBlocksQty","anonymousClassesQty",
             "innerClassesQty","lambdasQty",
             #"uniqueWordsQty", # Number of unique words in the source code
             "modifiers",
             #"logStatementsQty", # Number of log statements in the source code
             #"fix",
             "entrophy",
             "la","ld",
             #"net_lines_changed","absolute_lines_changed",
             "lines_per_file",
             "changed_file_count",
             #"entropy_bucket",
             ]

In [5]:
import pandas as pd
# read the merged df

df = pd.read_csv('merged_datasets/new/tomcat_merged_df.csv')

print(df.shape[0])

126165


In [6]:
# print column which has missing values
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])


d_lcom*            6607
d_tcc             22281
d_lcc             22281
lcom*              6589
tcc               22114
lcc               22114
classification    59600
dtype: int64


In [7]:
#print distinc values in tcc column
print(df['tcc'].unique())
# and their counts
print(df['tcc'].value_counts())

# print nan count
print(df['tcc'].isnull().sum())

[-1.  0. nan]
tcc
 0.0    93641
-1.0    10410
Name: count, dtype: int64
22114


# DECISON
should we delete columns with possible N/A values or delete rows with N/A values ? 

if number of visible methods in a class is less than 2, then CK prefers to set it -1.
For some reason, some rows have NaN values, so set it to -1 as well as it is not applicable.

In [8]:
def fix_df(df):
    """
    XGBoost supports missing values, so we don't need to impute them.
    """

    return df



    # Handle NA values
    """
    df['d_lcom*'] = df['d_lcom*'].fillna(-1)
    df['d_tcc'] = df['d_tcc'].fillna(-1)
    df['d_lcc'] = df['d_lcc'].fillna(-1)
    df['lcom*'] = df['lcom*'].fillna(-1)
    df['tcc'] = df['tcc'].fillna(-1)
    df['lcc'] = df['lcc'].fillna(-1)
    """
    # drop these columns
    # remove from X_COLUMNS
    
    try:
        X_COLUMNS.remove('d_lcom*')
        X_COLUMNS.remove('lcom*')
        X_COLUMNS.remove('d_tcc')
        X_COLUMNS.remove('tcc')
        X_COLUMNS.remove('d_lcc')
        X_COLUMNS.remove('lcc')
    except:
        # already removed
        pass
    
    df = df.drop(columns=['d_lcom*', 'lcom*', 'd_tcc', 'tcc', 'd_lcc', 'lcc'])
    return df

df = fix_df(df)

In [9]:
DATASETS = ["merged_datasets/new/broadleaf_merged_df.csv",
            "merged_datasets/new/camel_merged_df.csv",
            "merged_datasets/new/dubbo_merged_df.csv",
            "merged_datasets/new/elasticsearch_merged_df.csv",
            "merged_datasets/new/guava_merged_df.csv",
            "merged_datasets/new/jdk_merged_df.csv",
            "merged_datasets/new/jgroups_merged_df.csv",
            "merged_datasets/new/kafka_merged_df.csv",
            "merged_datasets/new/spark_merged_df.csv",
            "merged_datasets/new/spring-boot_merged_df.csv",
            "merged_datasets/new/spring-framework_merged_df.csv",
            "merged_datasets/new/tomcat_merged_df.csv",
            ]

In [10]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import ColorScaleRule

def write_formatted_report(ws, df, start_row, start_col):
    """
    Writes the DataFrame `df` into worksheet `ws` starting at (start_row, start_col).
    
    - The header row is formatted with bold text and a light gray fill.
    - For any row whose first cell (the "class" column) equals "accuracy"
      (case-insensitive), the function writes:
         • The label "accuracy" in the first column.
         • The accuracy value (assumed to be in the second value of the row)
           into the cell corresponding to the "f1-score" header.
         • All other numeric cells (except the support column) are left blank.
         • The "support" column (if present) is left unchanged.
    - After writing the table, a conditional color–scale is applied to all 
      numeric cells (i.e. all cells except the first "class" column and the 
      "support" column). Also, any empty cell in that range is cleared of fill.
      
    Float values are formatted with two decimals.
    
    Returns the total number of rows written (header + data).
    """
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    
    headers = list(df.columns)
    num_cols = len(headers)
    total_rows = df.shape[0] + 1  # header + data rows

    # Write the header row.
    for j, col_name in enumerate(headers, start=start_col):
        cell = ws.cell(row=start_row, column=j, value=col_name)
        cell.font = header_font
        cell.fill = header_fill

    # Write data rows.
    for i, row_data in enumerate(df.values, start=start_row + 1):
        # Check if this is the "accuracy" row.
        if str(row_data[0]).strip().lower() == "accuracy":
            # Write the "accuracy" label in the first column.
            ws.cell(row=i, column=start_col, value=row_data[0])
            # Find the column index for "f1-score" (case-insensitive) in headers.
            f1_idx = None
            for idx, h in enumerate(headers):
                if h.strip().lower() == "f1-score":
                    f1_idx = idx
                    break
            # Write the accuracy value into the "f1-score" column.
            if f1_idx is not None and len(row_data) > 1:
                accuracy_value = row_data[1]  # assume the accuracy value is in the second column
                cell = ws.cell(row=i, column=start_col + f1_idx, value=accuracy_value)
                if isinstance(accuracy_value, float):
                    cell.number_format = "0.00"
            # For the remaining columns:
            for j, h in enumerate(headers[1:], start=start_col + 1):
                # Skip the f1-score column (already written) and the support column.
                if (f1_idx is not None and j == start_col + f1_idx) or h.strip().lower() == "support":
                    continue
                ws.cell(row=i, column=j, value="")
        else:
            # Normal row: write each cell value.
            for j, value in enumerate(row_data, start=start_col):
                cell = ws.cell(row=i, column=j, value=value)
                if isinstance(value, float):
                    cell.number_format = "0.00"
    
    # Determine the range for numeric cells to apply conditional formatting.
    # We want to apply formatting to all numeric columns except:
    #   - the first column ("class")
    #   - the "support" column (if present)
    #
    # Find the index of "support" in headers (if it exists).
    support_idx = None
    for idx, h in enumerate(headers):
        if h.strip().lower() == "support":
            support_idx = idx
            break
    numeric_start_col = start_col + 1  # skip "class"
    if support_idx is not None:
        # Apply formatting to columns before the support column.
        numeric_end_col = start_col + support_idx - 1  # excludes the support column
    else:
        numeric_end_col = start_col + num_cols - 1

    numeric_start_row = start_row + 1
    numeric_end_row = start_row + df.shape[0]
    # Build range string (e.g. "B2:D10").
    cell_range = f"{get_column_letter(numeric_start_col)}{numeric_start_row}:{get_column_letter(numeric_end_col)}{numeric_end_row}"
    
    # Define a color scale: low values in red, mid in yellow, high in green.
    color_scale_rule = ColorScaleRule(
        start_type="num", start_value=0, start_color="F8696B",
        mid_type="num", mid_value=0.5, mid_color="FFEB84",
        end_type="num", end_value=1, end_color="63BE7B"
    )
    ws.conditional_formatting.add(cell_range, color_scale_rule)
    
    # Remove fill from any empty cells in the numeric range.
    for row in ws[cell_range]:
        for cell in row:
            if cell.value in [None, ""]:
                cell.fill = PatternFill(fill_type=None)
    
    return total_rows

###############################################################################
# Main export function.
###############################################################################
def export_results_to_excel(results, output_file):
    """
    Groups the classification reports (from the global dict `results`) by dataset
    and writes them all into a single worksheet.
    
    For each dataset, the random-split and date-split reports are written side by side,
    with a title row above each table (e.g. "Dataset1 random split" and "Dataset1 date split").
    Each dataset block is written on a new set of rows (separated by a blank gap).
    
    Finally, the workbook is saved to `output_file`.
    """
    # Group the reports by dataset name.
    # Expected keys: "<dataset_name>_random_split" and "<dataset_name>_date_split"
    grouped_results = {}
    for key, report in results.items():
        try:
            ds_name, split_type, _ = key.rsplit("_", 2)
        except ValueError:
            continue
        if ds_name not in grouped_results:
            grouped_results[ds_name] = {}
        grouped_results[ds_name][split_type] = report

    # Create a new workbook with one worksheet.
    wb = Workbook()
    ws = wb.active
    ws.title = "Classification Reports"
    
    current_row = 1  # starting row on the sheet
    gap_rows = 2     # blank rows between dataset blocks

    for ds_name, splits in grouped_results.items():
        # Convert the reports into DataFrames.
        if "random" in splits:
            df_random = pd.DataFrame(splits["random"]).T.reset_index().rename(columns={"index": "class"})
        else:
            df_random = None

        if "date" in splits:
            df_date = pd.DataFrame(splits["date"]).T.reset_index().rename(columns={"index": "class"})
        else:
            df_date = None

        # Determine starting column for the date table.
        if df_random is not None:
            left_table_cols = df_random.shape[1]
        else:
            left_table_cols = 0
        date_table_start_col = left_table_cols + 3 if df_random is not None else 1

        # Write title cells above each table.
        ws.cell(row=current_row, column=1, value=f"{ds_name} random split")
        ws.cell(row=current_row, column=date_table_start_col, value=f"{ds_name} date split")

        # Write the tables starting on the next row.
        start_table_row = current_row + 1
        rows_random = 0
        rows_date = 0
        if df_random is not None:
            rows_random = write_formatted_report(ws, df_random, start_table_row, 1)
        if df_date is not None:
            rows_date = write_formatted_report(ws, df_date, start_table_row, date_table_start_col)

        # Determine block height (title row + maximum table height).
        block_height = 1 + max(rows_random, rows_date)
        current_row += block_height + gap_rows

    wb.save(output_file)
    print(f"Workbook saved as {output_file}")


In [11]:
from sklearn.feature_selection import SelectFromModel
from mrmr import mrmr_classif

def select_features(x, y, method="model", estimator=None, threshold="median", 
                    correlation_threshold=0.9, num_features=50):
    """
    Select features using different methods.

    Parameters:
      - x: Training features (pandas DataFrame)
      - y: Training labels (pandas Series)
      - method: Which feature selection method to use.
                Options are:
                  "model" : Model-based feature selection using SelectFromModel.
                  "cbfs"  : Correlation-based feature selection (CBFS).
                  "mrmr"  : Minimum Redundancy Maximum Relevance (MRMR) using mrmr_selection.
      - estimator: (For "model" method) An estimator to compute feature importances.
                   If None, an XGBClassifier is used.
      - threshold: (For "model" method) The threshold for feature importance in SelectFromModel.
                   Default is "median".
      - correlation_threshold: (For "cbfs" method) Threshold to drop highly correlated features.
      - num_features: (For "mrmr" method) Number of features to select.
      
    Returns:
      - A list of selected feature names.
    """
    method = method.lower()
    if method == "model":
        if estimator is None:
            estimator = XGBClassifier(tree_method="hist", device="cuda", random_state=42)
        estimator.fit(x, y)
        selector = SelectFromModel(estimator, threshold=threshold, prefit=True)
        selected_features = list(x.columns[selector.get_support()])
        return selected_features

    elif method == "cbfs":
        # Compute the absolute correlation matrix and remove one of any pair of highly correlated features.
        corr_matrix = x.corr().abs()
        # Use only the upper triangle of the correlation matrix.
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
        selected_features = [col for col in x.columns if col not in to_drop]
        return selected_features

    elif method == "mrmr":
        # Call mrmr_classif directly with the DataFrame (not x.values) so that it can access the column names.
        selected_features = mrmr_classif(x, y, K=num_features)
        return selected_features

    else:
        raise ValueError("Method must be one of 'model', 'cbfs', or 'mrmr'")

In [12]:
results = {}
best_param_dict = {}

In [13]:
from sklearn.preprocessing import MinMaxScaler

def main(result_prefix: str, feature_selection_method="mrmr", n_features = 50, n_splits=5, n_trials=50, correlation_threshold=0.9):


    for dataset in DATASETS:

        dataset_name = dataset.split("new")[1][1:]

        CP = ["merged_datasets/new/spring-boot_merged_df.csv",
            "merged_datasets/new/spring-framework_merged_df.csv",
            "merged_datasets/new/tomcat_merged_df.csv",
            "merged_datasets/new/broadleaf_merged_df.csv",
            "merged_datasets/new/dubbo_merged_df.csv",
            ]

        for processed in results.keys():
            if dataset_name in processed:
                continue        

        print(f"Processing {dataset_name}")
        df = pd.read_csv(dataset)
        df = fix_df(df)

        df = pd.DataFrame()
        # read all datasets in CP into single df
        for cp_dataset in CP:
            print(cp_dataset)
            cp_df = pd.read_csv(cp_dataset)
            cp_df = fix_df(cp_df)
            df = pd.concat([df, cp_df])


        """
        TODO
        - It might not be best to split the dataset here. We should split the dataset in the hyperparameter optimization step.
        - We should also perform feature selection in the hyperparameter optimization step. MAYBE ???
        """
        

        # -------------------------
        # Random split approach
        # -------------------------
        x_train, x_test, y_train, y_test = train_test_split(df[X_COLUMNS + INFO_COLUMNS], df[Y_COLUMN],
                                                            test_size=0.2, random_state=42)
        
        # Scale after split to avoid leakage
        scaler = MinMaxScaler()
        x_train[X_COLUMNS] = scaler.fit_transform(x_train[X_COLUMNS])
        x_test[X_COLUMNS] = scaler.transform(x_test[X_COLUMNS])


        # Perform feature selection on the training set using only X_COLUMNS
        selected_features_random = select_features(x_train[X_COLUMNS], y_train,
                                                   method=feature_selection_method, num_features=n_features, correlation_threshold=correlation_threshold)
        print(f"Selected features for {dataset_name} (random split): {selected_features_random}")

        # Optimize hyperparameters on the selected features
        best_params = hyperparameter_optimization(x_train[selected_features_random], y_train, n_splits=n_splits, n_trials=n_trials)
        best_model = XGBClassifier(**best_params, tree_method="hist", device="cuda")
        best_model.fit(x_train[selected_features_random], y_train)
        y_pred = best_model.predict(x_test[selected_features_random])
        results[f"{dataset_name}_random_split"] = classification_report(y_test, y_pred, output_dict=True)
        best_param_dict[f"{dataset_name}_random_split"] = best_params

        # -------------------------
        # Date split approach
        # -------------------------

        # Lets not use first %20(time) of the dataset
        # as the project evolves over time
        # we will use remaining %80 percent as training + test set 

        """df_trimmed = df.sort_values(by='author_date_unix_timestamp').iloc[int(df.shape[0]*0.2):]

        ########################experimental"""
        
        x_train_date, x_test_date, y_train_date, y_test_date = train_test_split_by_date(
            df[X_COLUMNS + INFO_COLUMNS], df[Y_COLUMN], test_size=0.2)
        
        # Scale splits
        x_train_date[X_COLUMNS] = scaler.fit_transform(x_train_date[X_COLUMNS])
        x_test_date[X_COLUMNS] = scaler.transform(x_test_date[X_COLUMNS])

        selected_features_date = select_features(x_train_date[X_COLUMNS], y_train_date,
                                                 method=feature_selection_method, num_features=n_features, correlation_threshold=correlation_threshold)
        print(f"Selected features for {dataset_name} (date split): {selected_features_date}")

        best_params = hyperparameter_optimization(x_train_date[selected_features_date], y_train_date, n_splits=n_splits, n_trials=n_trials)
        best_model = XGBClassifier(**best_params, tree_method="hist", device="cuda")
        best_model.fit(x_train_date[selected_features_date], y_train_date)
        y_pred = best_model.predict(x_test_date[selected_features_date])
        results[f"{dataset_name}_date_split"] = classification_report(y_test_date, y_pred, output_dict=True)
        best_param_dict[f"{dataset_name}_date_split"] = best_params

        #####
        break

    # Export the classification reports to Excel.
    export_results_to_excel(results, output_file=f"{result_prefix}_classification_reports.xlsx")

    # save best params as json
    with open(f'{result_prefix}_best_params.json', 'w') as f:
        json.dump(best_param_dict, f)

In [14]:
main("cp_mrmr_50", feature_selection_method="mrmr", n_features=50, n_splits=5, n_trials=30, correlation_threshold=0.9)

Processing broadleaf_merged_df.csv
merged_datasets/new/spring-boot_merged_df.csv
merged_datasets/new/spring-framework_merged_df.csv
merged_datasets/new/tomcat_merged_df.csv
merged_datasets/new/broadleaf_merged_df.csv
merged_datasets/new/dubbo_merged_df.csv


100%|██████████| 50/50 [00:36<00:00,  1.36it/s]


Selected features for broadleaf_merged_df.csv (random split): ['entrophy', 'changed_file_count', 'la', 'ld', 'noc', 'd_dit', 'lambdasQty', 'anonymousClassesQty', 'tcc', 'd_defaultMethodsQty', 'd_protectedFieldsQty', 'finalFieldsQty', 'lines_per_file', 'fanout', 'd_publicMethodsQty', 'rfc', 'd_lcom*', 'lcc', 'innerClassesQty', 'cbo', 'd_anonymousClassesQty', 'd_cbo', 'loc', 'dit', 'nosi', 'd_fanout', 'comparisonsQty', 'd_visibleMethodsQty', 'maxNestedBlocksQty', 'd_defaultFieldsQty', 'd_stringLiteralsQty', 'variablesQty', 'privateFieldsQty', 'd_totalMethodsQty', 'privateMethodsQty', 'd_loc', 'staticMethodsQty', 'wmc', 'd_tcc', 'd_privateFieldsQty', 'loopQty', 'returnQty', 'd_totalFieldsQty', 'modifiers', 'numbersQty', 'd_maxNestedBlocksQty', 'assignmentsQty', 'd_wmc', 'd_cboModified', 'fanin']


[I 2025-02-19 12:39:23,584] A new study created in memory with name: no-name-cc919fd6-a296-41c0-813e-4313da2c1fea
[I 2025-02-19 12:39:32,964] Trial 0 finished with value: 0.8309135100937548 and parameters: {'n_estimators': 122, 'min_child_weight': 3.63475792691668, 'max_depth': 4, 'learning_rate': 0.01492558592304627, 'subsample': 0.6020237249396614, 'colsample_bytree': 0.7099969323498221, 'gamma': 5.520626348611435e-07, 'reg_alpha': 2.3679033996037874e-07, 'reg_lambda': 5.689693424301708e-08, 'scale_pos_weight': 14.176419874511167}. Best is trial 0 with value: 0.8309135100937548.
[I 2025-02-19 12:39:55,632] Trial 1 finished with value: 0.9439558212972423 and parameters: {'n_estimators': 180, 'min_child_weight': 1.1819512005976787, 'max_depth': 9, 'learning_rate': 0.13287352083244633, 'subsample': 0.7931879080672695, 'colsample_bytree': 0.536513792915912, 'gamma': 1.4636082810712613e-05, 'reg_alpha': 9.099592547224332e-08, 'reg_lambda': 9.068022771012491e-07, 'scale_pos_weight': 15.025

Selected features for broadleaf_merged_df.csv (date split): ['entrophy', 'd_dit', 'changed_file_count', 'ld', 'la', 'dit', 'lambdasQty', 'anonymousClassesQty', 'd_lcom*', 'lines_per_file', 'd_cbo', 'cbo', 'modifiers', 'd_publicMethodsQty', 'publicFieldsQty', 'd_fanout', 'fanout', 'noc', 'd_visibleMethodsQty', 'rfc', 'd_maxNestedBlocksQty', 'd_totalFieldsQty', 'tcc', 'd_cboModified', 'd_totalMethodsQty', 'staticMethodsQty', 'd_protectedFieldsQty', 'loc', 'd_privateFieldsQty', 'd_loc', 'finalFieldsQty', 'd_stringLiteralsQty', 'lcc', 'd_anonymousClassesQty', 'variablesQty', 'maxNestedBlocksQty', 'd_assignmentsQty', 'fanin', 'd_variablesQty', 'comparisonsQty', 'd_wmc', 'd_returnQty', 'loopQty', 'numbersQty', 'd_rfc', 'defaultMethodsQty', 'wmc', 'innerClassesQty', 'd_parenthesizedExpsQty', 'assignmentsQty']


[I 2025-02-19 12:51:56,223] Trial 0 finished with value: 0.8785564865827459 and parameters: {'n_estimators': 106, 'min_child_weight': 2.3016656924208854, 'max_depth': 9, 'learning_rate': 0.011106721800365218, 'subsample': 0.5048058640660962, 'colsample_bytree': 0.8779956212042725, 'gamma': 7.69407885439524e-07, 'reg_alpha': 1.232879859560591e-08, 'reg_lambda': 0.002208381590506238, 'scale_pos_weight': 19.348547877078385}. Best is trial 0 with value: 0.8785564865827459.
[I 2025-02-19 12:52:18,662] Trial 1 finished with value: 0.9421879407445587 and parameters: {'n_estimators': 277, 'min_child_weight': 4.952288700537578, 'max_depth': 7, 'learning_rate': 0.1901823522235316, 'subsample': 0.700144182332352, 'colsample_bytree': 0.6094043731184279, 'gamma': 2.928420138995144e-07, 'reg_alpha': 0.10853436143209655, 'reg_lambda': 0.010865359793076868, 'scale_pos_weight': 4.774715232768973}. Best is trial 1 with value: 0.9421879407445587.
[I 2025-02-19 12:52:29,266] Trial 2 finished with value: 0

Workbook saved as cp_mrmr_50_classification_reports.xlsx


In [14]:
"""export_results_to_excel(results, output_file="classification_reports.xlsx")

# save best params as json
with open('best_params.json', 'w') as f:
    json.dump(best_param_dict, f)"""

'export_results_to_excel(results, output_file="classification_reports.xlsx")\n\n# save best params as json\nwith open(\'best_params.json\', \'w\') as f:\n    json.dump(best_param_dict, f)'

In [15]:
pipelines = [
    # (result_prefix, feature_selection_method, n_features, n_splits, n_trials, correlation_threshold)
    #("mrmr100", "mrmr", 100, 5, 50, None), ## almost all columns
    #("mrmr50", "mrmr", 50, 5, 50, None),
    ("mrmr25", "mrmr", 25, 5, 50, None),
    #("mrmr10", "mrmr", 10, 5, 50, None),
    ("cbfs09", "cbfs", None, 5, 50, 0.9),
    ("cbfs05", "cbfs", None, 5, 50, 0.5),
    #("cbfs01", "cbfs", None, 5, 50, 0.1),
    #("model", "model", None, 5, 50, None),
]

from time import sleep

for result_prefix, feature_selection_method, n_features, n_splits, n_trials, correlation_threshold in pipelines:
    print(f"Running pipeline: {result_prefix}")
    main(result_prefix, feature_selection_method, n_features, n_splits, n_trials, correlation_threshold)
    sleep(300) # cool off for 5 minutes

Running pipeline: mrmr25
Processing broadleaf_merged_df.csv


100%|██████████| 25/25 [00:06<00:00,  4.14it/s]


Selected features for broadleaf_merged_df.csv (random split): ['entrophy', 'd_synchronizedMethodsQty', 'changed_file_count', 'ld', 'dit', 'la', 'd_dit', 'lines_per_file', 'd_cbo', 'defaultMethodsQty', 'loc', 'd_lcom*', 'd_cboModified', 'd_fanout', 'stringLiteralsQty', 'd_maxNestedBlocksQty', 'd_publicMethodsQty', 'modifiers', 'rfc', 'anonymousClassesQty', 'd_totalMethodsQty', 'd_totalFieldsQty', 'loopQty', 'd_visibleMethodsQty', 'd_staticMethodsQty']


[I 2025-02-17 21:05:35,290] A new study created in memory with name: no-name-1fa6174b-c0c3-4d96-bf97-30c51a879898
[I 2025-02-17 21:05:44,386] Trial 0 finished with value: 0.9805319028863163 and parameters: {'n_estimators': 197, 'min_child_weight': 3.5331531387740625, 'max_depth': 5, 'learning_rate': 0.06740349170323155, 'subsample': 0.651870757987133, 'colsample_bytree': 0.9856182769227608, 'gamma': 0.00011588411785553649, 'reg_alpha': 1.3486794822866283e-07, 'reg_lambda': 0.011175973618420846, 'scale_pos_weight': 15.754059262165946}. Best is trial 0 with value: 0.9805319028863163.
[I 2025-02-17 21:06:03,516] Trial 1 finished with value: 0.9933411221780577 and parameters: {'n_estimators': 300, 'min_child_weight': 1.485753389285414, 'max_depth': 9, 'learning_rate': 0.07023718184746083, 'subsample': 0.7806795779469105, 'colsample_bytree': 0.7729203783533543, 'gamma': 0.24570822616239213, 'reg_alpha': 2.6626478468362716e-07, 'reg_lambda': 2.706033516909091e-07, 'scale_pos_weight': 1.70386

Selected features for broadleaf_merged_df.csv (date split): ['entrophy', 'lines_per_file', 'changed_file_count', 'dit', 'ld', 'la', 'd_dit', 'd_cbo', 'd_lcom*', 'loc', 'd_fanout', 'abstractMethodsQty', 'd_cboModified', 'd_maxNestedBlocksQty', 'd_staticMethodsQty', 'rfc', 'd_visibleMethodsQty', 'innerClassesQty', 'd_totalMethodsQty', 'stringLiteralsQty', 'd_anonymousClassesQty', 'd_totalFieldsQty', 'loopQty', 'd_loc', 'd_publicMethodsQty']


...

Selected features for tomcat_merged_df.csv (random split): ['d_cbo', 'd_wmc', 'd_dit', 'd_noc', 'd_lcom', 'd_lcom*', 'd_tcc', 'd_staticMethodsQty', 'd_privateMethodsQty', 'd_protectedMethodsQty', 'd_defaultMethodsQty', 'd_abstractMethodsQty', 'd_finalMethodsQty', 'd_synchronizedMethodsQty', 'd_totalFieldsQty', 'd_privateFieldsQty', 'd_protectedFieldsQty', 'd_defaultFieldsQty', 'd_synchronizedFieldsQty', 'd_nosi', 'd_tryCatchQty', 'd_parenthesizedExpsQty', 'd_stringLiteralsQty', 'd_numbersQty', 'd_mathOperationsQty', 'd_maxNestedBlocksQty', 'd_anonymousClassesQty', 'd_innerClassesQty', 'd_modifiers', 'cbo', 'dit', 'noc', 'tcc', 'staticMethodsQty', 'defaultMethodsQty', 'abstractMethodsQty', 'finalMethodsQty', 'synchronizedMethodsQty', 'defaultFieldsQty', 'synchronizedFieldsQty', 'anonymousClassesQty', 'entrophy', 'la', 'lines_per_file']


[I 2025-02-18 09:31:25,044] Trial 0 finished with value: 0.6962988402117489 and parameters: {'n_estimators': 147, 'min_child_weight': 4.998566335859046, 'max_depth': 4, 'learning_rate': 0.0937343017995624, 'subsample': 0.5242049374691071, 'colsample_bytree': 0.6427728658089642, 'gamma': 0.0027883704056212008, 'reg_alpha': 0.0012165648619316398, 'reg_lambda': 0.022684369087718998, 'scale_pos_weight': 15.246185773000382}. Best is trial 0 with value: 0.6962988402117489.
[I 2025-02-18 09:31:32,469] Trial 1 finished with value: 0.7931163373691661 and parameters: {'n_estimators': 267, 'min_child_weight': 5.975913014620265, 'max_depth': 4, 'learning_rate': 0.046152658696839315, 'subsample': 0.8016855589785263, 'colsample_bytree': 0.6517269971207131, 'gamma': 0.0033667504657981743, 'reg_alpha': 0.0503506990883303, 'reg_lambda': 5.472497295457773e-08, 'scale_pos_weight': 9.086718631798943}. Best is trial 1 with value: 0.7931163373691661.
[I 2025-02-18 09:31:41,074] Trial 2 finished with value: 

Selected features for tomcat_merged_df.csv (date split): ['d_cbo', 'd_wmc', 'd_dit', 'd_noc', 'd_lcom', 'd_lcom*', 'd_tcc', 'd_staticMethodsQty', 'd_privateMethodsQty', 'd_protectedMethodsQty', 'd_defaultMethodsQty', 'd_abstractMethodsQty', 'd_finalMethodsQty', 'd_synchronizedMethodsQty', 'd_totalFieldsQty', 'd_privateFieldsQty', 'd_protectedFieldsQty', 'd_defaultFieldsQty', 'd_synchronizedFieldsQty', 'd_nosi', 'd_parenthesizedExpsQty', 'd_stringLiteralsQty', 'd_numbersQty', 'd_maxNestedBlocksQty', 'd_anonymousClassesQty', 'd_innerClassesQty', 'd_lambdasQty', 'd_modifiers', 'cbo', 'dit', 'noc', 'tcc', 'staticMethodsQty', 'defaultMethodsQty', 'abstractMethodsQty', 'finalMethodsQty', 'synchronizedMethodsQty', 'defaultFieldsQty', 'synchronizedFieldsQty', 'anonymousClassesQty', 'lambdasQty', 'entrophy', 'la', 'lines_per_file']


[I 2025-02-18 09:41:15,635] Trial 0 finished with value: 0.6901971755966566 and parameters: {'n_estimators': 282, 'min_child_weight': 1.6022342975819241, 'max_depth': 3, 'learning_rate': 0.09452617482754548, 'subsample': 0.9386741712720172, 'colsample_bytree': 0.7099320459206471, 'gamma': 0.0003384520496829929, 'reg_alpha': 0.0010036064141345773, 'reg_lambda': 0.4607201116312944, 'scale_pos_weight': 13.2358320529005}. Best is trial 0 with value: 0.6901971755966566.
[I 2025-02-18 09:41:22,101] Trial 1 finished with value: 0.9066149372197959 and parameters: {'n_estimators': 136, 'min_child_weight': 2.600834611853452, 'max_depth': 8, 'learning_rate': 0.11808435936403527, 'subsample': 0.7822830089734502, 'colsample_bytree': 0.6976021105793437, 'gamma': 0.024651401983081862, 'reg_alpha': 3.222550589318774e-07, 'reg_lambda': 8.392308949620614e-08, 'scale_pos_weight': 7.932458113854025}. Best is trial 1 with value: 0.9066149372197959.
[I 2025-02-18 09:41:27,179] Trial 2 finished with value: 0

Workbook saved as cbfs05_classification_reports.xlsx
